Aqui possuem duas etapas.

1- Aplicação do treinamento de ETL

2- No final realizei a incorporação do Desafio solicitado, pra avaliar
vá até o final.

#Extract
Extraindo a lista de IDs de usuario

Para analisar e testa direto no colab, acesse:https://colab.research.google.com/drive/167vSq-gojD_ZSBItq8ZzYJT_Ohax8cVP?usp=sharing

e solicite sue acesso.

In [6]:
#Utilizando a api do dev santander week
sdw2023_api = 'https://sdw-2023-prd.up.railway.app/'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import files
uploadfile = files.upload()

Saving BTCSTD2023.csv to BTCSTD2023.csv


In [5]:
import pandas as pd

df = pd.read_csv('BTCSTD2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[2051, 2052, 2053]


In [7]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api}users/{id}')
  return response.json() if response.status_code == 200 else None


users = [user for id in user_ids if (user := get_user(id)) is not None]

print(json.dumps(users, indent=2))

[
  {
    "id": 2051,
    "name": "string_Cesar",
    "account": {
      "id": 2150,
      "number": "x01x00x-2",
      "agency": "x0001",
      "balance": 10.0,
      "limit": 10.0
    },
    "card": {
      "id": 1983,
      "number": "**** 1122 **** 1122",
      "limit": 10.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 2052,
    "name": "string_Cesar2",
    "account": {
      "id": 2151,
      "number": "x01x00x2-2",
      "agency": "x20001",
      "balance": 10.0,
      "limit": 10.0
    },
    "card": {
      "id": 1984,
      "number": "**** 1X22 **** 1122",
      "limit": 10.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 2053,
    "name": "string_Cesar3",
    "account": {
      "id": 2152,
      "number": "x01x00x3-2",
      "agency": "x30001",
      "balance": 10.0,
      "limit": 10.0
    },
    "card": {
      "id": 1985,
      "number": "**** 1X32 **** 1122",
      "limit": 10.0
    },
    "features": [],
    "news": []
  }
]


#Transform
Utilizando a API do chatGPT para gerar uma mensaem do marketing personalizada para cda usuario

In [8]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.4 MB/s eta 0:00:00


In [34]:
#api key para treinamento do botcamp Santander 2023 criada em 10/09/2023 e sera deletado 30 dias corridos apos a entrega do desafio na plataforma
openai_api_key = 'sk-incluisuachave, poiseu alterei, tendo em vista que paguei com cartao de credito para conseguir realizar corretamente'

In [35]:
import openai
openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
      {
          "role": "system",
          "content": "Você é um especialista em marketing bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

In [36]:
for user in users:
  news = generate_ai_news(user)
  print(news)

Olá Cesar, investir é garantir um futuro tranquilo. Torne seus sonhos ativos com plano de investimentos.
Olá Cesar2! Invista no seu futuro, seu dinheiro merece render mais. Com nossas opções, é seguro e fácil.
Olá, Cesar3! Ganhe com gastos inteligentes! Seu dinheiro pode crescer com investimentos. Vamos resolver isso?


In [37]:
#Incluindo a mensagem na lista
for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append(
      {
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": news
      }
  )

Cesar, investir é crucial para aumentar sua riqueza, garantindo estabilidade e segurança futuras!
Cesar, invista no futuro! Crie segurança financeira e multiplique sua riqueza com os investimentos bancários.
Olá string_Cesar3, invista seu dinheiro! O futuro se constrói hoje. Transforme sonhos em realidade.


#Load
Atualiza a lista de dados na api da dev week

In [38]:
def update_user(user):
  response = requests.put(f"{sdw2023_api}users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

In [39]:
#testando o envio
for user in users:
  sucess = update_user(user)
  print(f"User{user['name']} updated? {sucess}!")

Userstring_Cesar updated? True!
Userstring_Cesar2 updated? True!
Userstring_Cesar3 updated? True!


#Até aqui, o projeto de aprendizado foi conlcuido com sucesso

#Desafio. Criando minha própria ETL usando API Alpha Vantage

A ideia é realizar uma tarefa simples, sem uso de banco de banco de dados
Neste desafio irei simuliar via arquivo parquet como se fosse uma atualização do banco de dados. Espero que aprecie o desafio

#Extração

In [40]:
#Instalando biblioteca do alpha vantage
!pip install alpha_vantage

In [41]:
# No contexto vou aplicar para uma extração de dados históricos financeiro utilizando API, afim de realizar analises e ate utilizar para criar modelos
#Importando a biblioteca
from alpha_vantage.timeseries import TimeSeries

#Chave de Acesso a API da Alpha Vantage
ALPHAVANTAGE_API_KEY= 'utilizesua apiKey, troquei depois de realizar todas as tarefas'

In [42]:
#Dados do Ativo para extração
#Utilizei apenas 1 unico ativo para testar a conexão,
#mas é possivel também, passar em lista, arquivo etc conforme ja foi ensinado
ativo = 'GBPUSD'

In [56]:
#Acessando a API
ts = TimeSeries(key=ALPHAVANTAGE_API_KEY, output_format='pandas')

In [57]:
df, meta_dados = ts.get_daily(symbol=ativo, outputsize='full')

In [58]:
#visualizando os dados carregados
df.head(10)

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2023-09-08,1.2473,1.2514,1.2451,1.2465,0.0
2023-09-07,1.2501,1.2508,1.2446,1.2471,0.0
2023-09-06,1.2565,1.2588,1.2482,1.2500,0.0
2023-09-05,1.2627,1.2632,1.2528,1.2565,0.0
2023-09-04,1.2588,1.2642,1.2586,1.2626,0.0
2023-09-01,1.2671,1.2713,1.2577,1.2586,0.0
2023-08-31,1.2719,1.2734,1.2652,1.2671,0.0
2023-08-30,1.2645,1.2746,1.2618,1.2718,0.0
2023-08-29,1.2600,1.2655,1.2562,1.2643,0.0


In [59]:
meta_dados

{'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
 '2. Symbol': 'GBPUSD',
 '3. Last Refreshed': '2023-09-08 21:55:00',
 '4. Output Size': 'Full size',
 '5. Time Zone': 'US/Eastern'}

#Transformando o dataset


In [60]:
df.columns

Index(['1. open', '2. high', '3. low', '4. close', '5. volume'], dtype='object')

In [61]:
df_dados_financeiro = df.reset_index()

In [62]:
df_dados_financeiro

,date,1. open,2. high,3. low,4. close,5. volume
0,2023-09-08,1.2473,1.2514,1.2451,1.2465,0.0
1,2023-09-07,1.2501,1.2508,1.2446,1.2471,0.0
2,2023-09-06,1.2565,1.2588,1.2482,1.2500,0.0
3,2023-09-05,1.2627,1.2632,1.2528,1.2565,0.0
4,2023-09-04,1.2588,1.2642,1.2586,1.2626,0.0
...,...,...,...,...,...,...
4995,2004-07-15,1.8508,1.8573,1.8473,1.8524,0.0
4996,2004-07-14,1.8559,1.8621,1.8528,1.8559,0.0
4997,2004-07-13,1.8566,1.8658,1.8514,1.8572,0.0
4998,2004-07-12,1.8600,1.8665,1.8563,1.8642,0.0


In [63]:
#ajustando o cabeçalho
df_dados_financeiro.rename(columns={
    'date':'DATE', '1. open':'OPEN', '2. high':'HIGH', '3. low':'LOW', '4. close':'CLOSE', '5. volume':'VOLUME'
}, inplace=True)

#Excluindo a feature VOLUME, pois, no momento não será relevante para analise.
df_dados_financeiro.drop(columns=['VOLUME'], inplace=True)

df_dados_financeiro.head(10)

,DATE,OPEN,HIGH,LOW,CLOSE
0,2023-09-08,1.2473,1.2514,1.2451,1.2465
1,2023-09-07,1.2501,1.2508,1.2446,1.2471
2,2023-09-06,1.2565,1.2588,1.2482,1.2500
3,2023-09-05,1.2627,1.2632,1.2528,1.2565
4,2023-09-04,1.2588,1.2642,1.2586,1.2626
5,2023-09-01,1.2671,1.2713,1.2577,1.2586
6,2023-08-31,1.2719,1.2734,1.2652,1.2671
7,2023-08-30,1.2645,1.2746,1.2618,1.2718
8,2023-08-29,1.2600,1.2655,1.2562,1.2643
9,2023-08-28,1.2580,1.2611,1.2565,1.2600


In [64]:
#visulizando informações do dataset
df_dados_financeiro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATE    5000 non-null   datetime64[ns]
 1   OPEN    5000 non-null   float64       
 2   HIGH    5000 non-null   float64       
 3   LOW     5000 non-null   float64       
 4   CLOSE   5000 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 195.4 KB


In [ ]:
#Os dados estao normalizado e estao prontos para serem salvos

#Carregamento no BD simulado pelo arquivo Parquet

In [ ]:
#Salvando as extrações
#Devido ao projeto de teste, para evitar aumentar muito codigo
#Considerei que o projeto extrai todas as informações e salve em arquivo
#parquet, tendo em vista que o parquet mantém o Dtype de acordo com o Dataframe

In [65]:
df_dados_financeiro.to_parquet('/content/BTCSTD2023.parquet', compression='gzip')